In [97]:
M = 16

In [98]:
import re

import numpy as np
import pandas as pd

In [99]:
from tqdm.autonotebook import tqdm

## Importing Datasets

In [100]:
string_df = pd.read_csv("../../data/interim/string_df.csv", index_col=0)

balanced_pairs_df = pd.read_csv("../../data/train_test/test_pairs.csv", index_col=0)

balanced_pairs_df.drop_duplicates(inplace=True)
balanced_pairs_df.reset_index(drop=True, inplace=True)

In [101]:
string_df

,concatenated
label,
GooglePixel3A_L,0000000000000000000000000000000000000000000000...
GooglePixel3A_L,0000000000000000000000000000000000000000000000...
GooglePixel3A_L,0000000000000000000000000000000000000000000000...
GooglePixel3A_L,0000000000000000000000000000000000000000000000...
GooglePixel3A_L,0000000000000000000000000000000000000000000000...
...,...
iPhoneXSMax_M,0001101000101101000000000001101111111111000000...
iPhoneXSMax_M,0001101000101101010000000001101111111111000000...
iPhoneXSMax_M,0001101000101101000000000001101111111111000000...


In [102]:
string_df["concatenated"] = string_df["concatenated"].apply(
    lambda x: np.array(list(x)).astype(int)
)

## Importing Best Configurations

In [103]:
import re
import pandas as pd


def parse_log_file(filename):
    data = []

    with open(filename, "r") as file:
        lines = file.readlines()

        current_filter = None
        current_threshold = None
        current_min_error = None
        current_confidence = None

        for line in lines:
            if "Best Filter" in line:
                # Extract Best Filter using regex
                filter_match = re.search(r"Best Filter: (.+)", line)
                if filter_match:
                    current_filter = filter_match.group(1).strip()

            elif "Best Threshold" in line:
                # Extract Best Threshold using regex
                threshold_match = re.search(r"Best Threshold: (.+)", line)
                if threshold_match:
                    current_threshold = int(threshold_match.group(1).strip())

            elif "Min error" in line:
                # Extract Min Error using regex
                min_error_match = re.search(r"Min error: (.+)", line)
                if min_error_match:
                    current_min_error = float(min_error_match.group(1).strip())

            elif "Confidence" in line:
                # Extract Confidence using regex
                confidence_match = re.search(r"Confidence: (.+)", line)
                if confidence_match:
                    current_confidence = float(confidence_match.group(1).strip())

                    # Once we have all values, create a tuple and add it to the data list
                    data.append(
                        (
                            current_filter,
                            current_threshold,
                            current_min_error,
                            current_confidence,
                        )
                    )

                    # Reset current values for the next entry
                    current_filter = None
                    current_threshold = None
                    current_min_error = None
                    current_confidence = None

    # Convert the list of tuples into a DataFrame
    df = pd.DataFrame(
        data, columns=["Best Filter", "Best Threshold", "Min Error", "Confidence"]
    )

    return df

In [104]:
# Usage example:
filename = "../../reports/best_config"
best_configs_df = parse_log_file(filename)

In [105]:
best_configs_df

,Best Filter,Best Threshold,Min Error,Confidence
0,0[304] 🀆🀆🀆🀆🀆🀆🀆🀆🀫🀫🀫🀫🀫🀫🀫🀫 0[1464],1,0.453672,0.185844
1,0[1752] 🀆🀆🀆🀆🀫🀫🀫🀫 0[24],1,0.459875,0.160845
2,0[328] 🀫🀫🀫🀫🀆🀆🀆🀆 0[1448],1,0.460774,0.157226
3,0[304] 🀫🀫🀫🀫🀫🀫🀫🀫🀫🀫🀫🀫🀫🀫🀫🀫 0[1464],2,0.461820,0.153019
4,0[1664] 🀫🀫🀫🀫🀫🀫🀫🀫🀫🀫🀫🀫🀫🀫🀫🀫 0[104],1,0.459515,0.162296
...,...,...,...,...
109,0[16] 🀫🀫🀫🀫🀆🀆🀆🀆 0[1760],1,0.288404,0.903146
110,0[472] 🀆🀆🀆🀆🀫🀫🀫🀫 0[1304],1,0.288404,0.903146
111,0[976] 🀫🀫🀫🀫🀆🀆🀆🀆 0[800],1,0.288404,0.903146
112,0[152] 🀫🀫🀫🀫🀆🀆🀆🀆 0[1624],1,0.288404,0.903146


In [106]:
if M != 0:
    best_configs_df = best_configs_df.head(M)

if M == 0:
    M = len(best_configs_df)

In [107]:
compression_rate = len(string_df["concatenated"].iloc[0]) / best_configs_df.shape[0]

print("Compression Rate:", compression_rate)

Compression Rate: 111.5


## Generate Matrix

## Filters Parser

In [108]:
def filter_parser(input_string: str) -> list:
    # Split the string into its parts
    parts = input_string.split()

    # Initialize the final array
    result = []

    # Process each part
    for part in parts:
        if part.startswith("0["):
            # Extract the number inside the brackets
            count = int(part[2:-1])
            # Append the corresponding number of zeros to the result
            result.extend([0] * count)
        else:
            # Translate the tiles to their respective values
            for char in part:
                if char == "🀆":
                    result.append(-1)
                elif char == "🀫":
                    result.append(1)

    return result

In [109]:
# import time

# time.sleep(100)

## Apply Filters

In [110]:
def apply_filter(item, filter):
    # item = np.array(list(item)).astype(int)
    item = item.astype(int)
    filter = filter_parser(filter)
    return np.sum(np.multiply(item, filter))

In [111]:
def apply_filter_threshold(item, filter, threshold) -> int:
    if apply_filter(item, filter) > threshold:
        return 1
    else:
        return -1

In [112]:
def apply_filter_threshold_pair(item_1, item_2, filter, threshold) -> int:
    if apply_filter_threshold(item_1, filter, threshold) == apply_filter_threshold(
        item_2, filter, threshold
    ):
        return 1
    else:
        return -1

## Calculate Fingerprint

In [113]:
def hamming_distance(array1, array2, confidence):
    # Check if arrays have the same length
    if len(array1) != len(array2):
        raise ValueError("Arrays must have the same length")

    # Initialize distance counter
    distance = 0

    # Iterate through arrays and count differences
    for i in range(len(array1)):
        if array1[i] != array2[i]:
            distance += confidence[i]

    distance = (distance / sum(confidence)) * len(confidence)

    return distance

In [114]:
def calculate_fingerprint(item, best_filters, best_thresholds, confidence):
    fingerprint = []

    for best_filter, best_threshold in zip(best_filters, best_thresholds):
        filtered = np.sum(np.multiply(item.astype(int), filter_parser(best_filter)))

        if filtered > best_threshold:
            filtered = 1
        else:
            filtered = -1

        fingerprint.append(filtered)

    return fingerprint

In [115]:
fingerprints = []

In [116]:
for i, row in tqdm(string_df.iterrows(), total=string_df.shape[0]):
    # Extracting best filters and thresholds from the main DataFrame
    best_filters = best_configs_df["Best Filter"].tolist()
    best_thresholds = best_configs_df["Best Threshold"].tolist()
    confidence = best_configs_df["Confidence"].tolist()

    # Calculate the fingerprint using the relevant best filters and thresholds
    fingerprint = calculate_fingerprint(
        row["concatenated"], best_filters, best_thresholds, confidence
    )

    # Store the result in the 'fprint' column
    # string_df.at[i, "fprint"] = fingerprint
    fingerprints.append(fingerprint)

  0%|          | 0/956 [00:00<?, ?it/s]

In [117]:
string_df["fprint"] = fingerprints

## Clustering w/ Cosine Similarity

In [118]:
string_df = string_df.reset_index()

In [119]:
string_df

,label,concatenated,fprint
0,GooglePixel3A_L,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, -1, 1, 1, -1, 1, 1, -1, -1, 1, -1..."
1,GooglePixel3A_L,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, -1, 1, 1, -1, -1, 1, -1,..."
2,GooglePixel3A_L,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, -1, 1, -1, 1, 1, -1, 1, 1, -1, -1, 1, -..."
3,GooglePixel3A_L,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, -1, 1, 1, -1, -1, 1, -1,..."
4,GooglePixel3A_L,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, -1, 1, 1, -1, 1, 1, -1, -1, 1, -1..."
...,...,...,...
951,iPhoneXSMax_M,"[0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, ...","[-1, 1, -1, -1, -1, -1, 1, -1, 1, 1, 1, -1, -1..."
952,iPhoneXSMax_M,"[0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, ...","[-1, -1, -1, -1, -1, -1, -1, 1, -1, -1, 1, -1,..."
953,iPhoneXSMax_M,"[0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, ...","[-1, -1, -1, -1, -1, -1, 1, -1, -1, 1, 1, -1, ..."
954,iPhoneXSMax_M,"[0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, ...","[-1, 1, -1, -1, 1, -1, 1, 1, 1, 1, 1, -1, -1, ..."


In [120]:
len(string_df["fprint"].iloc[0])

16

In [121]:
def cosine_similarity(arr1, arr2):
    """
    Compute cosine similarity between two arrays.

    Parameters:
    arr1 : numpy array
        First array.
    arr2 : numpy array
        Second array.

    Returns:
    float
        Cosine similarity between arr1 and arr2.
    """
    dot_product = np.dot(arr1, arr2)
    norm_arr1 = np.linalg.norm(arr1)
    norm_arr2 = np.linalg.norm(arr2)
    similarity = dot_product / (norm_arr1 * norm_arr2)

    return similarity

In [122]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import homogeneity_score, completeness_score, v_measure_score

# Example dataframe setup (replace this with your actual dataframe)
# string_df = pd.DataFrame({'label': labels, 'fprint': fingerprints, 'concatenated_fprint': concatenated_fingerprints})

# Extract arrays from dataframe
data_points = string_df["fprint"].values
labels_true = string_df["label"].values


# Compute cosine similarity between each pair of data points
def compute_cosine_similarity_matrix(data_points):
    num_points = len(data_points)
    similarity_matrix = np.zeros((num_points, num_points))

    for i in range(num_points):
        for j in range(i, num_points):
            # Reshape arrays to ensure they are 2D
            array1 = np.array(data_points[i]).reshape(1, -1)
            array2 = np.array(data_points[j]).reshape(1, -1)
            similarity = cosine_similarity(array1, array2)[0, 0]
            similarity_matrix[i, j] = similarity
            similarity_matrix[j, i] = similarity

    return similarity_matrix


similarity_matrix = compute_cosine_similarity_matrix(data_points)

# Example clustering (replace this with your actual clustering algorithm)
# Here, clustering assignment is based on a cosine similarity threshold
clusters = []
cluster_indices = np.zeros(len(data_points), dtype=int)

threshold = 0.65

current_cluster_index = 0
for i in range(len(data_points)):
    if cluster_indices[i] == 0:
        cluster_indices[i] = current_cluster_index
        clusters.append([i])

        for j in range(i + 1, len(data_points)):
            if similarity_matrix[i, j] >= threshold:
                if cluster_indices[j] == 0:
                    cluster_indices[j] = current_cluster_index
                    clusters[current_cluster_index].append(j)

        current_cluster_index += 1

In [123]:
# Calculate evaluation metrics
homogeneity = homogeneity_score(labels_true, cluster_indices)
completeness = completeness_score(labels_true, cluster_indices)
v_measure = v_measure_score(labels_true, cluster_indices)

print(f"Homogeneity: {homogeneity:.4f}")
print(f"Completeness: {completeness:.4f}")
print(f"V-measure: {v_measure:.4f}")

Homogeneity: 0.6641
Completeness: 0.7225
V-measure: 0.6921


In [124]:
len(string_df["label"].unique())

33

In [125]:
len(set(cluster_indices))

28